### 데이터에서 이상치 감지&제거, 평균 값 대체

In [ ]:
class Anomaly:
    def simplema(self, df, col, window): # 주어진 df에서 특정 col의 이동 평균(SMA) 계산
        return df[col].rolling(window=window, min_periods=1).mean() # min_periods=1 ▶ 데이터 부족해도 최소 1개 있을 때 평균 계산
    
    def zscore(self, df, col, window, thresh=3, return_all=False):
        roll = df[col].rolling(window=window, min_periods=1, center=True) 
        avg = roll.mean()
        std = roll.std(ddof=0)
        # zscore 계산
        z = df[col].sub(avg).div(std)
        # zscore가 임계값 내부에 있는지 확인
        m = z.between(-thresh, thresh)
        
        if return_all:
            return z, avg, std, m
        return df[col].where(m, avg)
    
    def boxplot(self, df, col, window, return_all=False):
        roll = df[col].rolling(window=window, min_periods=1, center=True)
        avg = roll.mean()
        q1 = roll.quantile(0.25)
        q3 = roll.quantile(0.75)
        # 사분위수 계산
        iqr = q3 - q1
        # fence 설정
        fence_low = q1-1.5*iqr
        fence_high = q3+1.5*iqr
        m = df[col].between(fence_low, fence_high)
        
        if return_all:
            return df[col], avg, m
        return df[col].where(m, avg)